### Our main objective for this code is to find segments that vary in payment amounts/behaviour the assumption being that each has a stable segment level performance, which is a reasonable assumption because we have cohorts that are the same with respect to the segment/model score.

We are working with synthetic data, so there are distributions and such that doesn't follow the norm for real data, despite this - building the scaffolding for creating a CLV will still be pertinent.

Below is a class that loads the synthetic data, and maps the columns CustomerID,TransactionDate,TransactionAmount to the correct name. There are also methods in this class for creating RFM measures, RFM Segments, Demographics and Transaction Descriptor features (which I dont use), if you can do something useful with it then by all means.

Please Note that the dataloader, and rfm methods are general methods they can be used with any data that contains the columns CustomerID,TransactionID,TransactionDate,TransactionAmount.

I will close off by saying that the point of this exercise is to end off with segments that have stable performance at the "crowd" level. Thus nullifying the problem of high variance when trying to do a regression.

Once we have these segments, we have a choice of 2, we will then extrapolate payments per cohort/crowd/group. That will be the next step, this work will be carried out in a seperate notebook and class with written for the Vintage object with optimisation methods.

In [53]:
import pandas as pd
from pathlib import Path
import rfmbinner
import xgboost as xgb
import optuna
import numpy as np
import sklearn


class DataLoader:
    
    def __init__(self,path,customer_id,transaction_id,transaction_date,amount):
        """
        Initialize the DataLoader.

        Parameters:
            path (str): Path to the CSV file.
            customer_id (str): Column name for customer IDs.
            transactionid (str): Column name for transaction IDs.
            transaction_date (str): Column name for transaction dates.
            amount (str): Column name for transaction amounts.

        """
        self.path = path
        self.customer_id = customer_id
        self.transaction_id = transaction_id
        self.transaction_date = transaction_date 
        self.amount = amount 
    
    def fetch_data(self) -> pd.DataFrame:
        """
        Fetch raw CSV data from the specified path.

        Parameters:
            path (str): Path to the CSV file.

        Returns:
            pd.DataFrame: Loaded data.
        """
        data_path = Path(self.path)
        if not data_path.exists():
            raise FileNotFoundError(f"CSV file not found at: {data_path}")
        data = pd.read_csv(data_path)
        data.rename(columns={self.customer_id:'CustomerID',
                             self.transaction_id: 'TransactionID',
                             self.transaction_date: 'Date',
                             self.amount: 'Amount'}, inplace=True)
        data['Date'] = pd.to_datetime(data['Date'],format='%Y-%m-%d %H:%M:%S')
        return data
    
    def calculate_rfm(self, snapshot_date: str, window: pd.Timedelta,df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate RFM metrics for customer segmentation.

        Parameters:
            snapshot_date (str): Date to calculate recency from (YYYY-MM-DD).

        Returns:
            pd.DataFrame: RFM metrics per CustomerID.
        """
        data = df.copy()
        snapshot = pd.to_datetime(snapshot_date)
        data['Date'] = pd.to_datetime(data.Date)
        data  = data[(data.Date<snapshot) & (data.Date>(snapshot-window))]
        data['recency'] = (snapshot - pd.to_datetime(data['Date'])).dt.days
        print(data)
        rfm = data.groupby('CustomerID').agg({
            'recency': 'min',
            'TransactionID': 'count',               # frequency: number of transactions
            'Amount': 'sum'             # monetary: total spend
        }).reset_index()

        rfm.rename(columns={'TransactionID': 'frequency', 'Amount': 'monetary'}, inplace=True)
        rfm['date'] = snapshot_date

        return rfm[['CustomerID', 'date', 'recency', 'frequency', 'monetary']]
    
    def calculate_target(self,date: str, window_size: pd.Timedelta, \
                         repurchase_threshold: float, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculating the target for the model
        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            window_size (pd.Timedelta): Number of days in which to consider repurchase
            repurchase_threshold: Minimum spend to be considered a repurchase
        Returns:
            pd.DataFrame: Target per CustomerID.
        """
        #caculate whether a customer made total purchases after date exceeding repurchase threshold
        data = df.copy()
        future_purchases = data[(data['Date'] > date) & \
                                (data['Date'] <= date + window_size)]

        target_customers = future_purchases.groupby('CustomerID')[['Amount']].sum()
        target_customers = target_customers[target_customers >= repurchase_threshold].index

        total_future_purchases = future_purchases.groupby('CustomerID')[['Amount']].sum()
        data = data.merge(total_future_purchases.rename(columns={'Amount': 'subsequent_purchases'}),
                            on='CustomerID', how='left')
        data['subsequent_purchases'] = data['subsequent_purchases'].fillna(0)


        data['target'] = data['CustomerID'].isin(target_customers).astype(int)
        data['date'] = date
        return data[['CustomerID','date','target','subsequent_purchases']].drop_duplicates()
    
    def rfm_segments(self,date: str, window: pd.Timedelta, df: pd.DataFrame) -> pd.DataFrame:
        """
        Segments customers based on their RFM score.

        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            df (pd.DataFrame): Raw transactional data.

        Returns:
            pd.DataFrame: RFM segments per CustomerID.
        """
        data = df.copy()
        data = self.calculate_rfm(date,window,data)
        # Create RFM scores
        data['r_score'] = pd.qcut(data['recency'].rank(method='first'), 5, labels=[5, 4, 3, 2, 1])
        data['f_score'] = pd.qcut(data['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['m_score'] = pd.qcut(data['monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['rfm_score'] = data['r_score'].astype(str) + \
                            data['f_score'].astype(str) + \
                            data['m_score'].astype(str)
        data['rfm_score_int'] = data['r_score'].astype(int) + \
                            data['f_score'].astype(int)+ \
                            data['m_score'].astype(int)           
        #create a dicitionary with the rfm_score that maps into segment, the key must be a regex
        segment_map = {
                    r'^55[1-5]$': 'Champions',
                    r'^54[1-5]$': 'Loyal Customers',
                    r'^45[1-5]$': 'Potential Loyalists',
                    r'^53[1-5]$': 'New or Returning Customers',
                    r'^33[1-5]$': 'Promising',
                    r'^22[1-5]$': 'Needs Attention',
                    r'^\d{3}$': 'Others'  # Matches any other 3-digit score
                }
        data['segment'] = data['rfm_score'].replace(segment_map, regex=True)
        data['date'] = date
        return data[['CustomerID', 'date', 'rfm_score','rfm_score_int', 'segment']]
    
    def dedup_demographic_variables(self,df: pd.DataFrame) -> pd.DataFrame:
        """
        Deduplicate demographic variables for each customer, keeping the earliest entry.

        Parameters:
            df (pd.DataFrame): Raw transactional data with demographic information.

        Returns:
            pd.DataFrame: Deduplicated demographic data per CustomerID.
        """
        data = loader.fetch_data()
        data['Date'] = pd.to_datetime(data['Date'])
        
        # Sort by CustomerID and Date to get the most recent demographic info
        data = data.sort_values(by=['CustomerID', 'Date'], ascending=True)
        
        # Drop duplicates, keeping the last (most recent) entry for each CustomerID
        # Assuming demographic variables are 'Gender', 'Age', 'Age Group', extend to the actual list
        demographic_cols = ['CustomerID', 'Gender', 'Age','Province']
        deduplicated_demographics = data[demographic_cols].drop_duplicates(subset=['CustomerID'], keep='first')
        
        return deduplicated_demographics
    
    def transaction_descriptor_variables(self,date) -> pd.DataFrame:
        """
        Caclulates mode transaction dimensions purchased from each customer
        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
        Returns:
            pd.DataFrame: Most frequent transaction descriptors per CustomerID.
        """

        data = self.fetch_data()
        data['Date'] = pd.to_datetime(data['Date'])
        purchases = data[data.Date < date]
        summary = purchases.groupby('CustomerID')[['ProductCategory','PurchaseChannel',
                                       'PaymentMethod','Store']].agg(pd.Series.mode).reset_index()
        summary['date'] = date
        # Rename columns for clarity
        summary.rename(columns={'ProductCategory': 'Most_frequented_Category',
                                  'PurchaseChannel': 'Most_frequented_Channel',
                                  'PaymentMethod': 'Most_used_payment_method',
                                  'Store': 'Most_frequented_Store'}, inplace=True)
        return summary[['CustomerID','date','Most_frequented_Channel','Most_frequented_Category','Most_used_payment_method','Most_frequented_Store']]
    
    def dedup_demographic_variables(self,df: pd.DataFrame) -> pd.DataFrame:
        """
        Deduplicate demographic variables for each customer, keeping the latest entry.

        Parameters:
            df (pd.DataFrame): Raw transactional data with demographic information.

        Returns:
            pd.DataFrame: Deduplicated demographic data per CustomerID.
        """
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'],format='%m-%d-%Y %H:%M:%S')
        
        # Sort by CustomerID and Date to get the most recent demographic info
        data = data.sort_values(by=['CustomerID', 'Date'], ascending=True)
        
        # Drop duplicates, keeping the last (most recent) entry for each CustomerID
        # Assuming demographic variables are 'Gender', 'Age', 'Age Group', extend to the actual list
        demographic_cols = ['CustomerID', 'Gender', 'Age','Province']
        deduplicated_demographics = data[demographic_cols].drop_duplicates(subset=['CustomerID'], keep='last')
        
        return deduplicated_demographics


For the code below the process is as follows:

1. Initialise a dataloader
2. Fetch the synthetic data
3. Run the appropriate methods of the dataloader in order to create a collection of features
4. We join all the features together and create a "model_data" dataframe.

In [54]:
params = {'path':r'data\customer_transaction_data.csv',
            'customer_id':'CustomerID',
            'transaction_id':'TransactionID',
            'transaction_date':'PurchaseDate',
            'amount':'TotalAmount'}
loader = DataLoader(**params)
data = loader.fetch_data()

In [55]:
start_date = '2022-08-30'
end_date = pd.to_datetime('2025-09-26')
pivot_date = pd.to_datetime(start_date)
rfms = []
targets = []
rfm_segments = []
transaction_descriptor_data = []

while pivot_date<end_date:
    rfms.append(loader.calculate_rfm(pivot_date,pd.Timedelta(days=99999),data))
    rfm_segments.append(loader.rfm_segments(pivot_date,pd.Timedelta(days=99999),data))
    targets.append(loader.calculate_target(pivot_date,pd.Timedelta(weeks=12),0,data))
    pivot_date+=pd.Timedelta(weeks=4)

rfm_data = pd.concat(rfms)
target_data = pd.concat(targets)
rfm_segments = pd.concat(rfm_segments)
dem_data = loader.dedup_demographic_variables(data)

       TransactionID         CustomerID  ProductID                Date  \
167              168            laura68   13847173 2022-08-27 10:09:09   
195              196           xsanchez   90029905 2022-08-19 22:37:57   
312              313  andersonalexander   29954728 2022-08-13 09:44:18   
381              382         jimmurillo   59211433 2022-08-27 01:54:14   
387              388            david16   10308158 2022-08-27 14:25:59   
...              ...                ...        ...                 ...   
19826          19827       laurenthomas   82931490 2022-08-11 20:07:44   
19832          19833     jeffreywatkins   53784919 2022-08-18 00:06:32   
19944          19945     albertjohnston   79273152 2022-08-24 23:00:29   
19963          19964          kenneth04    1529784 2022-08-12 18:27:31   
19966          19967           bcompton   47780071 2022-08-24 03:33:48   

      ProductCategory  Quantity  PricePerUnit  Age  Gender  \
167                 c         4         90.15   3

In [56]:
len(data)

20000

In [57]:
model_data = rfm_data.merge(target_data,how='left',left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(dem_data,how='left',left_on='CustomerID',right_on='CustomerID')
#model_data = model_data.merge(transaction_descriptor_data,how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(rfm_segments,how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])

In [58]:
end_date_of_training = '2024-07-31'
model_data = model_data.set_index(['CustomerID','date']).drop('rfm_score',axis=1)
model_data = model_data[model_data.index.get_level_values(1)<end_date_of_training]

In [59]:
model_data.target.value_counts()/len(model_data)

target
0    0.983064
1    0.016936
Name: count, dtype: float64

In [60]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    model_data.drop(['target', 'subsequent_purchases'], axis=1),
    model_data['target'],
    test_size=0.2,
    random_state=42
)

for col in X_train.columns:
    sample = X_train[col].iloc[0]
    if isinstance(sample, (list, np.ndarray)):
        print(f"Column '{col}' contains unhashable types: {type(sample)}")

encoder = ce.TargetEncoder()
X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)

# Optuna + XGBoost training
def train_xgboost_with_optuna(X, y, n_trials=50, test_size=0.2, random_state=42):
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    def objective(trial):
        params = {
            "verbosity": 0,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "eta": trial.suggest_float("eta", 0.01, 0.3),
        }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        model = xgb.train(params, dtrain, num_boost_round=100,
                          evals=[(dval, "validation")],
                          early_stopping_rounds=10,
                          verbose_eval=False)
        preds = model.predict(dval)
        return sklearn.metrics.roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    best_params.update({
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "logloss"
    })

    final_model = xgb.train(best_params, xgb.DMatrix(X, label=y), num_boost_round=study.best_trial.number)
    return final_model, study

# Train and evaluate
model, study = train_xgboost_with_optuna(X_train, y_train)
y_probs = model.predict(xgb.DMatrix(X_test))

print(sklearn.metrics.classification_report(y_test, y_probs > 0.5))
print("ROC AUC:", sklearn.metrics.roc_auc_score(y_test, y_probs))


[I 2025-08-10 14:25:52,084] A new study created in memory with name: no-name-3ec28f28-f4b3-4384-bcba-3fdd862486f9
[I 2025-08-10 14:25:55,385] Trial 0 finished with value: 0.7267396892899204 and parameters: {'booster': 'dart', 'lambda': 8.953904971429567e-08, 'alpha': 0.0509126994985438, 'subsample': 0.5122580237138574, 'colsample_bytree': 0.6149341292589328, 'max_depth': 6, 'eta': 0.14143647406030968}. Best is trial 0 with value: 0.7267396892899204.
[I 2025-08-10 14:25:55,651] Trial 1 finished with value: 0.8011448225527179 and parameters: {'booster': 'gbtree', 'lambda': 3.121973903626259e-06, 'alpha': 0.0025135240814952355, 'subsample': 0.7228477837372538, 'colsample_bytree': 0.7577751769468337, 'max_depth': 8, 'eta': 0.108786776468217}. Best is trial 1 with value: 0.8011448225527179.
[I 2025-08-10 14:25:58,692] Trial 2 finished with value: 0.7878767432360135 and parameters: {'booster': 'dart', 'lambda': 2.702823728586604e-07, 'alpha': 0.0005253638532025695, 'subsample': 0.62593782910

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     32569
           1       0.61      0.03      0.06       567

    accuracy                           0.98     33136
   macro avg       0.80      0.51      0.52     33136
weighted avg       0.98      0.98      0.98     33136

ROC AUC: 0.85460923201822


In [61]:
sklearn.metrics.roc_auc_score(y_test,model.predict(xgb.DMatrix(X_test)))

0.85460923201822

In [62]:
X_test['QSegment'] = pd.qcut(y_probs,10)

In [63]:
summary_data = X_test.merge(target_data,how='left', left_on=['CustomerID','date'], right_on=['CustomerID','date'])
summary = summary_data.groupby(['QSegment']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

C:\Users\wesle\AppData\Local\Temp\ipykernel_53448\1106211645.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = summary_data.groupby(['QSegment']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})


,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
QSegment,,,,,,
"(-0.0009453, 0.00152]",3314,0.36%,"R 354,433.22",7.74%,R 0.45,"R 1,477.82"
"(0.00152, 0.00257]",3314,0.30%,"R 396,724.50",8.66%,R 0.58,"R 1,934.87"
"(0.00257, 0.00373]",3313,0.30%,"R 412,090.07",8.99%,R 0.42,"R 1,375.04"
"(0.00373, 0.00501]",3314,0.51%,"R 432,892.26",9.45%,R 0.86,"R 2,858.15"
"(0.00501, 0.00659]",3313,0.42%,"R 445,675.66",9.73%,R 0.54,"R 1,798.46"
"(0.00659, 0.00866]",3314,0.39%,"R 454,338.84",9.92%,R 0.41,"R 1,350.88"
"(0.00866, 0.0117]",3313,0.51%,"R 460,900.79",10.06%,R 0.38,"R 1,272.50"
"(0.0117, 0.017]",3314,0.97%,"R 483,776.03",10.56%,R 1.43,"R 4,732.53"
"(0.017, 0.0298]",3313,2.05%,"R 505,024.20",11.02%,R 2.60,"R 8,602.57"


In [64]:
summary_data = X_test.merge(target_data,how='left', left_on=['CustomerID','date'], right_on=['CustomerID','date'])
summary = summary_data.groupby(['rfm_score_int']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
rfm_score_int,,,,,,
3,260,0.38%,"R 4,806.79",0.10%,R 0.43,R 112.45
4,839,1.67%,"R 25,433.99",0.56%,R 1.86,"R 1,562.86"
5,1685,1.25%,"R 75,945.98",1.66%,R 1.74,"R 2,933.03"
6,2800,1.61%,"R 175,367.44",3.83%,R 2.53,"R 7,078.78"
7,4016,1.27%,"R 349,570.79",7.63%,R 1.48,"R 5,947.07"
8,4814,1.50%,"R 508,663.18",11.10%,R 2.30,"R 11,077.84"
9,4903,1.47%,"R 640,877.07",13.99%,R 2.25,"R 11,048.76"
10,4395,1.32%,"R 666,069.58",14.54%,R 2.03,"R 8,932.16"
11,3722,1.80%,"R 669,331.02",14.61%,R 2.56,"R 9,531.50"


In [65]:
pd.DataFrame(data = zip(X_train.columns,model.get_score(importance_type='gain').values()),columns=['Features','Importances'])

,Features,Importances
0,recency,2.999489
1,frequency,4.597369
2,monetary,4.968842
3,Gender,3.456574
4,Age,3.278572
5,Province,3.880691
6,rfm_score_int,3.255265
7,segment,3.737944


In [66]:
model_data.segment.unique()

array(['Others', 'Needs Attention', 'Promising',
       'New or Returning Customers', 'Loyal Customers', 'Champions',
       'Potential Loyalists'], dtype=object)